In [ ]:
!curl -L https://www.dropbox.com/s/rndzndlgpw3coiq/kaggle-download.zip -o kaggle-download.zip
!unzip kaggle-download.zip -d kaggle_data


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17    0    17    0     0     57      0 --:--:-- --:--:-- --:--:--    58
100   340  100   340    0     0    557      0 --:--:-- --:--:-- --:--:--   557
100   534    0   534    0     0    529      0 --:--:--  0:00:01 --:--:--     0
100  515M  100  515M    0     0  69.0M      0  0:00:07  0:00:07 --:--:-- 88.0M23  118M    0     0  40.1M      0  0:00:12  0:00:02  0:00:10 82.6M
Archive:  kaggle-download.zip
replace kaggle_data/metaData_taxistandsID_name_GPSlocation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [22]:
import pandas as pd
from datetime import datetime
import torch


In [23]:
!pip install --upgrade torch torchvision

  Using cached torch-2.0.1-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchvision-0.15.2-cp39-cp39-manylinux1_x86_64.whl (6.0 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached triton-2.0.0-1-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached 

# DATA STUFF

'A’ if this trip was dispatched from the central

‘B’ if this trip was demanded directly to a taxi driver on a specific stand

‘C’ otherwise (i.e. a trip demanded on a random street)

In [3]:
train = pd.read_csv('kaggle_data/train.csv')
train['ELAPSED'] = train['POLYLINE'].apply(lambda x: max(len(x.split('[')) - 2, 0) * 15)
train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",345
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",285
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",975
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",645
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",435


In [4]:
test = pd.read_csv('kaggle_data/test_public.csv')
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


In [5]:
train.groupby('CALL_TYPE').count()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED
CALL_TYPE,,,,,,,,,
A,364770,364770,0,364770,364770,364770,364770,364770,364770
B,817881,0,806579,817881,817881,817881,817881,817881,817881
C,528019,0,0,528019,528019,528019,528019,528019,528019


If you have call type A, you have origin call
If you have call type B, you have origin stand
If you have call type C, you have neither

In [6]:
def parse_time(x):
  # We are using python's builtin datetime library
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 

train[["YR", "MON", "DAY", "HR", "WK"]] = train[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED,YR,MON,DAY,HR,WK
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",345,2013,7,1,0,0
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",285,2013,7,1,0,0
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",975,2013,7,1,0,0
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",645,2013,7,1,0,0
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",435,2013,7,1,0,0


In [7]:
from sklearn.preprocessing import OneHotEncoder

call_type_encoder = OneHotEncoder(sparse=False, drop='first')
month_encoder = OneHotEncoder(sparse=False, drop='first')
wk_encoder = OneHotEncoder(sparse=False, drop='first')
hr_encoder = OneHotEncoder(sparse=False, drop='first')

In [8]:
encoded_call = call_type_encoder.fit_transform(train[['CALL_TYPE']])
call_columns = call_type_encoder.get_feature_names(['CALL_TYPE'])
train_oh = pd.concat([train, pd.DataFrame(encoded_call, columns=call_columns)], axis=1)

encoded_month = month_encoder.fit_transform(train[['MON']])
month_columns = month_encoder.get_feature_names(['MON'])
encoded_wk = wk_encoder.fit_transform(train[['WK']])
wk_columns = wk_encoder.get_feature_names(['WK'])
encoded_hr = hr_encoder.fit_transform(train[['HR']])
hr_columns = hr_encoder.get_feature_names(['HR'])

new_train_oh = pd.concat([train_oh, pd.DataFrame(encoded_month, columns=month_columns)], axis=1)
new_train_oh = pd.concat([new_train_oh, pd.DataFrame(encoded_wk, columns=wk_columns)], axis=1)
new_train_oh = pd.concat([new_train_oh, pd.DataFrame(encoded_hr, columns=hr_columns)], axis=1)

In [9]:
new_train_ftrs = new_train_oh.drop(columns=['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'TAXI_ID', 'TIMESTAMP', 'DAY_TYPE', 
                                            'MISSING_DATA', 'POLYLINE', 'YR', 'MON', 'DAY', 'HR', 'WK', 'ORIGIN_STAND']).fillna(0)
new_train_ftrs.head()

,ELAPSED,CALL_TYPE_B,CALL_TYPE_C,MON_2,MON_3,MON_4,MON_5,MON_6,MON_7,MON_8,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
0,345,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,285,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,975,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,645,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,435,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
test[["YR", "MON", "DAY", "HR", "WK"]] = test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
test_encoded_call = call_type_encoder.transform(test[['CALL_TYPE']])
test_call_columns = call_type_encoder.get_feature_names(['CALL_TYPE'])
test_oh = pd.concat([test, pd.DataFrame(test_encoded_call, columns=test_call_columns)], axis=1)

test_encoded_month = month_encoder.transform(test[['MON']])
test_month_columns = month_encoder.get_feature_names(['MON'])
test_encoded_wk = wk_encoder.transform(test[['WK']])
test_wk_columns = wk_encoder.get_feature_names(['WK'])
test_encoded_hr = hr_encoder.transform(test[['HR']])
test_hr_columns = hr_encoder.get_feature_names(['HR'])

new_test_oh = pd.concat([test_oh, pd.DataFrame(test_encoded_month, columns=test_month_columns)], axis=1)
new_test_oh = pd.concat([new_test_oh, pd.DataFrame(test_encoded_wk, columns=test_wk_columns)], axis=1)
new_test_oh = pd.concat([new_test_oh, pd.DataFrame(test_encoded_hr, columns=test_hr_columns)], axis=1)

In [11]:
new_test_ftrs = new_test_oh.drop(columns=['CALL_TYPE', 'ORIGIN_CALL', 'TAXI_ID', 'TIMESTAMP', 'DAY_TYPE', 
                                            'MISSING_DATA', 'YR', 'MON', 'DAY', 'HR', 'WK', 'ORIGIN_STAND']).fillna(0)
new_test_ftrs.head()

,TRIP_ID,CALL_TYPE_B,CALL_TYPE_C,MON_2,MON_3,MON_4,MON_5,MON_6,MON_7,MON_8,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
0,T1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,T2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,T3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,T4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,T5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# train_ftrs = train_oh[['ORIGIN_CALL', 'ORIGIN_STAND',  'CALL_TYPE_B', 'CALL_TYPE_C', 'YR', 'MON', 'DAY', 'HR', 'WK', 'ELAPSED']].fillna(-1)
# train_ftrs = train_oh[['CALL_TYPE_B', 'CALL_TYPE_C', 'ORIGIN_STAND', 'MON', 'DAY', 'HR', 'ELAPSED']].fillna(-1)

# train_ftrs.head()

# Model

In [12]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

In [13]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

device(type='cuda')

In [14]:
from sklearn.model_selection import train_test_split

bs = 64

X = new_train_ftrs.drop(columns='ELAPSED')
y = new_train_ftrs['ELAPSED']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

num_ftrs = X_train.shape[1]

train_tn = torch.tensor(X_train.values, dtype=torch.float32).view(len(X_train), 1, num_ftrs).to(device)
exp_tn = torch.tensor(y_train.values, dtype=torch.float32).view(train_tn.shape[0], 1, 1).to(device)

val_tn = torch.tensor(X_val.values, dtype=torch.float32).view(len(X_val), 1, num_ftrs).to(device)
val_exp_tn = torch.tensor(y_val.values, dtype=torch.float32).view(val_tn.shape[0], 1, 1).to(device)

input_data = list(zip(train_tn, exp_tn))

val_data = list(zip(val_tn, val_exp_tn))

trainloader = torch.utils.data.DataLoader(input_data, batch_size=bs, shuffle=True)
valloader = torch.utils.data.DataLoader(val_data, batch_size=bs, shuffle=True)

In [15]:
train_tn[0]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [63]:
# from sklearn.model_selection import train_test_split

# bs = 64

# X = train_ftrs.drop(columns='ELAPSED')
# y = train_ftrs['ELAPSED']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# num_ftrs = X_train.shape[1]

# train_tn = torch.tensor(X_train.values, dtype=torch.float32).view(len(X_train), 1, num_ftrs).to(device)
# exp_tn = torch.tensor(y_train.values, dtype=torch.float32).to(device)

# val_tn = torch.tensor(X_val.values, dtype=torch.float32).view(len(X_val), 1, num_ftrs).to(device)
# val_exp_tn = torch.tensor(y_val.values, dtype=torch.float32).to(device)

# input_data = list(zip(train_tn, exp_tn))
# input_data[0]

# val_data = list(zip(val_tn, val_exp_tn))

# trainloader = torch.utils.data.DataLoader(input_data, batch_size=bs, shuffle=True)
# valloader = torch.utils.data.DataLoader(val_data, batch_size=bs, shuffle=True)

In [11]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)

        # Pass the input sequence through the RNN layer
        out, hidden = self.rnn(x, hidden)

        # Reshape the output to match the linear layer's input shape
        out = out.contiguous().view(-1, self.hidden_size)

        # Pass the output through the linear layer
        out = self.fc(out)

        return out

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros
        hidden = torch.zeros(1, batch_size, self.hidden_size).to(device)
        return hidden


In [26]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.2, reg_lambda=0.001):
        super(FNN, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.regularizer = nn.L1Loss()

        self.reg_lambda = reg_lambda

    def forward(self, x):
        x = self.fc1(x)
#         x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
#         x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)

        return x


In [27]:
# Define hyperparameters
input_size = num_ftrs  # number of features
hidden_size = 100  # play around with and test 
output_size = 1  # dimension of output which is 1 elapsed time 

# model instatiation
# model = RNNModel(input_size, hidden_size, output_size).to(device)
model = FNN(input_size, hidden_size, output_size, dropout_rate=0.4).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
# criterion = nn.SmoothL1Loss()

optimizer_adam = optim.Adam(model.parameters(), lr=0.0001)
# optimizer_sgd = optim.SGD(model.parameters(), lr=0.0001)

In [31]:
num_epochs = 10

t_loss = []
v_loss = []
t_loss_ep = []
v_loss_ep = []


# Training loop
for epoch in tqdm(range(num_epochs)):
    # Set the model in training mode
    model.train()
    for i, data in enumerate(trainloader):
        # print(i)
        inp, exp = data
    
        # Forward pass
        eps = .0000001
        outputs = model(inp)
        
        # Now doing RMSE
        loss = torch.sqrt(criterion(outputs, exp) + eps)
#         loss = criterion(outputs, exp)

        # Backward pass and optimization
        optimizer_adam.zero_grad()
        loss.backward()
        optimizer_adam.step()
        
        # grab training loss every n batches
        if i % 40 == 0:
            t_loss.append(loss.item())

    # Validation loop
    model.eval()
    for i, data in enumerate(valloader):
        inp, exp = data
        with torch.no_grad():
            val_outputs = model(inp)
            val_loss = torch.sqrt(criterion(val_outputs, exp) + eps)
        if i % 10 == 0:
            v_loss.append(val_loss.item())
            
        
    # Print the loss for each epoch
    print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}") 
    t_loss_ep.append(loss.item())
    v_loss_ep.append(val_loss.item())



 10%|█         | 1/10 [00:50<07:33, 50.38s/it]

Epoch: 1, Training Loss: 338.6935119628906, Validation Loss: 392.71636962890625


 20%|██        | 2/10 [01:38<06:32, 49.11s/it]

Epoch: 2, Training Loss: 353.5575866699219, Validation Loss: 486.3640441894531


 30%|███       | 3/10 [02:29<05:49, 49.97s/it]

...

Epoch: 8, Training Loss: 358.52374267578125, Validation Loss: 426.80145263671875


 90%|█████████ | 9/10 [07:31<00:49, 49.42s/it]

Epoch: 9, Training Loss: 251.70797729492188, Validation Loss: 651.808349609375


100%|██████████| 10/10 [08:24<00:00, 50.46s/it]

Epoch: 10, Training Loss: 980.1962280273438, Validation Loss: 544.58056640625


In [50]:
for i, data in enumerate(trainloader):
        # print(i)
        inp, exp = data
    
        # Forward pass
        eps = 1e-6
        outputs = model(inp)

In [53]:
outputs.unique()

tensor([682.0498, 684.8121, 685.1355, 685.1392, 685.1393, 685.1414, 691.7851],
       device='cuda:0', grad_fn=<Unique2Backward>)

In [100]:
first_predictions = predictions

In [29]:
test_tn = torch.tensor(new_test_ftrs.drop(columns='TRIP_ID').values, dtype=torch.float32).view(len(new_test_ftrs), 1, num_ftrs).to(device)
testloader = torch.utils.data.DataLoader(test_tn, batch_size=16, shuffle=False)

model.eval()
predictions = []
for i, data in enumerate(testloader):
    with torch.no_grad():
        outputs = model(data)
    predictions.extend(outputs.squeeze().cpu().numpy())
    
submission = pd.DataFrame({'TRIP_ID': new_test_ftrs['TRIP_ID'], 'TRAVEL_TIME': predictions})
submission.to_csv('submission.csv', index=False)

In [30]:
predictions

[748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 832.1363,
 748.20123,
 832.1363,
 748.20123,
 748.20123,
 748.20123,
 854.52216,
 854.52216,
 854.52216,
 854.52216,
 854.52216,
 854.52216,
 854.52216,
 748.20123,
 854.52216,
 748.20123,
 832.1363,
 832.1363,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 854.52216,
 854.52216,
 854.52216,
 854.52216,
 832.1363,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 854.52216,
 854.52216,
 748.20123,
 748.20123,
 748.20123,
 854.52216,
 854.52216,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 748.20123,
 854.52216,
 832.1363,
 748.20123,
 854.52216,
 854.52216,
 832.1363,
 748.20123,
 748.20123,
 748.20123,
 832.1363,
 854.52216,
 748.20123,
 828.6816,
 854.52216,
 748.20123,
 854.52216,
 700.31647,
 806.63727,
 700.31647,
 806.63727,
 806.63727,
 806.63727,
 784.25146,
 700.31647,
 784.25146,
 784.25146,
 